# URL Structure

> Visualizing a website's URL structure with a treemap

In [ ]:
# | default_exp url_structure

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
import advertools as adv
import adviz
import pandas as pd

pd.options.display.max_colwidth = 120
import plotly.express as px

In [ ]:
# | export
_texttemplate = "<b>%{label} </b><br><br>Directory: <b>%{parent}/%{label}</b><br>Count: %{value:,}<br>%{percentParent:.1%} of %{parent}<br>"

In [ ]:
# | export
def url_structure(
    url_list,
    items_per_level=10,
    height=600,
    width=None,
    template="none",
    domain="example.com",
    title="URL Structure",
    subtitle=None,
):
    """
    Create a treemap for the first two URL path directories `example.com/dir_1/dir_2/`.

    Parameters
    ----------
    url_list : list
        Any list-like object with a bunch of URLs.
    items_per_level : int
        The number of items to display for each level of the treemap. All other
        items will be grouped under a special item called "Others".
    height : int
        The height of the chart in pixels.
    width : int
        The width of the chart in pixels.
    template : str
        Name of template to use for the chart.
    domain : str
        The main domain of the URL list. This will be displayed at the top
        panel in the treemap to display values like a breadcrumb.
    title : str
        The title of the figure. You can use/include the following HTML tags in
        the title: `<a>`, `<b>`, `<br>`, `<i>`, `<sub>`, `<sup>`
    subtitle : str
        The subtitle of the figure.

    Returns
    -------
    url_structure_treemap : plotly.graph_objects.Figure
    """
    urldf = adv.url_to_df(url_list)
    dir1_top_n = urldf["dir_1"].value_counts().head(items_per_level).index.tolist() + [
        "Others:"
    ]
    urldf["dir_1_clean"] = [x if x in dir1_top_n else "Others:" for x in urldf["dir_1"]]
    top_n_df = pd.DataFrame(dir1_top_n, columns=["dir_1_top_n"])

    dir2_valcounts = []

    for top_n in top_n_df["dir_1_top_n"]:
        tempdf = urldf[urldf["dir_1_clean"].eq(top_n)]
        valcountsdf = adviz.value_counts_plus(
            tempdf["dir_2"], show_top=items_per_level, style=False
        )
        valcountsdf = valcountsdf.assign(dir_1_value=top_n)
        dir2_valcounts.append(valcountsdf)
    dir2_df = pd.concat(dir2_valcounts, ignore_index=True)[
        ["dir_2", "count", "dir_1_value"]
    ]
    treemap_df = pd.merge(
        top_n_df, dir2_df, left_on="dir_1_top_n", right_on="dir_1_value", how="left"
    )
    fig = px.treemap(
        treemap_df.dropna(),
        path=[px.Constant(domain), "dir_1_top_n", "dir_2"],
        branchvalues="total",
        maxdepth=2,
        width=width,
        height=height,
        title=title,
        subtitle=subtitle,
        template=template,
        values="count",
    )
    fig.data[0].marker.line.width = 0.01
    fig.data[0].marker.pad = dict.fromkeys("lrbt", 0)
    fig.data[0]["texttemplate"] = _texttemplate
    fig.data[0]["hovertemplate"] = _texttemplate
    fig.update_traces(pathbar={"edgeshape": "/"})
    return fig

## Read a list of URLs from a text/CSV file {.smaller}

In [ ]:
# | hide
import os

import pandas as pd


In [ ]:
# | hide
if os.getcwd().endswith("/nbs"):
    filepath = "data/apple_url_list.csv"
else:
    filepath = "nbs/data/apple_url_list.csv"

In [ ]:
apple = pd.read_csv(filepath)
apple.head(10)

,url
0,https://www.apple.com/ae/shop/accessories/all
1,https://www.apple.com/ae/shop/accessories/all/...
2,https://www.apple.com/ae/shop/accessories/all/...
3,https://www.apple.com/ae/shop/accessories/all/...
4,https://www.apple.com/ae/shop/accessories/all/...
5,https://www.apple.com/ae/shop/accessories/all/...
6,https://www.apple.com/ae/shop/accessories/all/...
7,https://www.apple.com/ae/shop/accessories/all/...
8,https://www.apple.com/ae/shop/accessories/all/...
9,https://www.apple.com/ae/shop/accessories/all/...


## Visualize the URL structure with {.smaller}

In [ ]:
# | echo: true
url_structure(apple["url"])

## Number of items per level {.smaller}

In [ ]:
# | echo: true
# | code-line-numbers: "3"
url_structure(url_list=apple["url"], items_per_level=5)

## Number of items per level {.smaller}

In [ ]:
# | echo: true
# | code-line-numbers: "3"
url_structure(url_list=apple["url"], items_per_level=25)

## Pick a template {.smaller}

In [ ]:
# | echo: true
# | code-line-numbers: "4"
url_structure(url_list=apple["url"], items_per_level=25, template="plotly_dark")

## Pick a template {.smaller}

In [ ]:
# | echo: true
# | code-line-numbers: "4"
url_structure(url_list=apple["url"], items_per_level=15, template="seaborn")

## Set domain name and chart title {.smaller}

In [ ]:
# | echo: true
# | code-fold: true
# | code-line-numbers: "5,6"
url_structure(
    url_list=apple["url"],
    items_per_level=15,
    template="ggplot2",
    domain="apple.com",
    title='URL Structure: <b>apple.com</b><br>Raw data: <a href="data/apple_url_list.csv">Apple.com URLs</a>',
)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()